# ALGORITMOS BASADOS EN SERIES TEMPORALES

## 5) Modelo SIMPLE - Promedio de Venta Simple (PVS))

Por lo relevado con los compradores es la forma en la que hoy compran. Toman la venta de los últimos 30 dias, la ven separada en 2 quincenas. Aplican criterio en base a los quiebres que haya tenido durante los últimos 30 dias. Exluye los datos de quiebre para calcular el promedio. Si tuvo oferta, le baja un poco la demanda. Si va a tener oferta la sube un poco. Toda esa información futura no está en el sistema.

1) **Preprocesamiento**
* Convertir la columna Fecha a formato datetime (si aún no lo está).
* Seleccionar las columnas necesarias y ordenar los datos cronológicamente.

2) **Agrupación y Resampleo**
* Agrupar los datos por Codigo_Articulo y Sucursal.
* Resamplear las ventas a frecuencia diaria para obtener la suma de Unidades vendidas por día.

3) **Cálculo del Pronóstico**
* Para cada grupo (artículo y sucursal), calcular la media diaria de los últimos N días (por ejemplo, 30 días).
* Multiplicar esta media por el número de días de la ventana de reposición (30 o 45 días) para obtener la cantidad a solicitar.

4) **Salida y Exportación**

* Generar un DataFrame con columnas: Codigo_Articulo, Sucursal y Cantidad (total a pedir).
* Exportar este DataFrame a un archivo CSV para su uso en el siguiente proceso.


In [30]:
# LIBRERIAS NECESARIAS 
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pyodbc
from dotenv import dotenv_values

# Mostrar el DataFrame resultante
import ace_tools_open as tools

# Evitar Mensajes Molestos
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category= FutureWarning)

In [ ]:
# FUNCIONES
  
def Open_Connection():
    secrets = dotenv_values(".env")   # Connection String from .env
    conn_str = f'DRIVER={secrets["DRIVER2"]};SERVER={secrets["SERVIDOR2"]};PORT={secrets["PUERTO2"]};DATABASE={secrets["BASE2"]};UID={secrets["USUARIO2"]};PWD={secrets["CONTRASENA2"]}'
    # print (conn_str) 
    try:    
        conn = pyodbc.connect(conn_str)
        return conn
    except:
        print('Error en la Conexión')
        return None
    
def Close_Connection(conn): 
    conn.close()
    return True

def generar_datos(id_proveedor, etiqueta):
    print(f"Generando datos para ID: {id_proveedor}, Label: {etiqueta}")
    # Configuración de conexión
    conn = Open_Connection()
    
    # FILTRA solo PRODUCTOS HABILITADOS y Traer datos de STOCK y PENDIENTES desde PRODUCCIÓN
    query = f"""
    SELECT  A.[C_PROVEEDOR_PRIMARIO]
        ,S.[C_ARTICULO]
        ,S.[C_SUCU_EMPR]
        ,S.[Q_FACTOR_VENTA_ESP]
        ,S.[Q_FACTOR_VTA_SUCU]
        ,S.[M_OFERTA_SUCU]
        ,S.[M_HABILITADO_SUCU]
        ,A.M_BAJA
        ,S.[Q_VTA_DIA_ANT]
        ,S.[Q_VTA_ACUM]
        ,S.[Q_ULT_ING_STOCK]
        ,S.[Q_STOCK_A_ULT_ING]
        ,S.[Q_15DIASVTA_A_ULT_ING_STOCK]
        ,S.[Q_30DIASVTA_A_ULT_ING_STOCK]
        ,S.[Q_BULTOS_PENDIENTE_OC]
        ,S.[Q_PESO_PENDIENTE_OC]
        ,S.[Q_UNID_PESO_PEND_RECEP_TRANSF]
        ,S.[Q_UNID_PESO_VTA_MES_ACTUAL]
        ,S.[F_ULTIMA_VTA]
        ,S.[Q_VTA_ULTIMOS_15DIAS]
        ,S.[Q_VTA_ULTIMOS_30DIAS]
        ,S.[Q_TRANSF_PEND]
        ,S.[Q_TRANSF_EN_PREP]
        ,S.[M_FOLDER]
        ,S.[M_ALTA_RENTABILIDAD]
        ,S.[Lugar_Abastecimiento]
        ,S.[M_COSTO_LOGISTICO]
    
    FROM [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
    LEFT JOIN [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
        ON A.[C_ARTICULO] = S.[C_ARTICULO]

    WHERE S.[M_HABILITADO_SUCU] = 'S' -- Permitido Reponer
        AND A.M_BAJA = 'N'  -- Activo en Maestro Artículos
        AND A.[C_PROVEEDOR_PRIMARIO] = {id_proveedor} -- Solo del Proveedor
    ;
    """
    # Ejecutar la consulta SQL
    articulos = pd.read_sql(query, conn)
    
    # Consulta SQL para obtener las ventas de un proveedor específico   
    # Reemplazar {proveedor} en la consulta con el ID de la tienda actual
    query = f"""
    SELECT V.[F_VENTA] as Fecha
        ,V.[C_ARTICULO] as Codigo_Articulo
        ,V.[C_SUCU_EMPR] as Sucursal
        ,V.[I_PRECIO_VENTA] as Precio
        ,V.[I_PRECIO_COSTO] as Costo
        ,V.[Q_UNIDADES_VENDIDAS] as Unidades
        ,V.[C_FAMILIA] as Familia
        ,A.[C_RUBRO] as Rubro
        ,A.[C_SUBRUBRO_1] as SubRubro
        ,LTRIM(RTRIM(REPLACE(REPLACE(REPLACE(A.N_ARTICULO, CHAR(9), ''), CHAR(13), ''), CHAR(10), ''))) as Nombre_Articulo
        ,A.[C_CLASIFICACION_COMPRA] as Clasificacion
    FROM [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T702_EST_VTAS_POR_ARTICULO] V
    LEFT JOIN [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T050_ARTICULOS] A 
        ON V.C_ARTICULO = A.C_ARTICULO
    WHERE A.[C_PROVEEDOR_PRIMARIO] = {id_proveedor} AND V.F_VENTA >= '20210101' AND A.M_BAJA ='N'
    ORDER BY V.F_VENTA ;
    """

    # Ejecutar la consulta SQL
    demanda = pd.read_sql(query, conn)
    
    # UNIR Y FILTRAR solo la demanda de los Hartículos VALIDOS.
    # Realizar la unión (merge) de los DataFrames por las claves especificadas
    data = pd.merge(
        articulos,  # DataFrame de artículos
        demanda,    # DataFrame de demanda
        left_on=['C_ARTICULO', 'C_SUCU_EMPR'],  # Claves en 'articulos'
        right_on=['Codigo_Articulo', 'Sucursal'],  # Claves en 'demanda'
        how='inner'  # Solo traer los productos que están en 'articulos'
    )
    
    # Guardar los resultados en un archivo CSV con el nombre del Proveedor
    file_path = f'data/{etiqueta}.csv'
    data.to_csv(file_path, index=False, encoding='utf-8')
    print(f"Datos de Venta guardados: {file_path}")
    
    file_path = f'data/{etiqueta}_articulos.csv'
    articulos.to_csv(file_path, index=False, encoding='utf-8')
    
    print(f"Archivos guardados: {file_path}")
    
    # Eliminar Columnas Innecesarias
    data = data[['Fecha', 'Codigo_Articulo', 'Sucursal', 'Unidades']]

    # Cerrar la conexión después de la iteración
    Close_Connection(conn)
    return data, articulos


def Calcular_Demanda(df_prv,forecast_window,id_proveedor, etiqueta):
    # Lista para almacenar los resultados del pronóstico
    resultados = []

    # Agrupar los datos por 'Codigo_Articulo' y 'Sucursal'
    for (codigo, sucursal), grupo in df_prv.groupby(['Codigo_Articulo', 'Sucursal']):
        # Establecer 'Fecha' como índice y ordenar los datos
        grupo = grupo.set_index('Fecha').sort_index()
        
        # Resamplear a diario sumando las ventas
        ventas_diarias = grupo['Unidades'].resample('D').sum().fillna(0)
        
        # Seleccionar un periodo reciente para calcular la media; por ejemplo, los últimos 30 días
        # Si hay menos de 30 días de datos, se utiliza el periodo disponible
        ventas_recientes = ventas_diarias[-30:]
        media_diaria = ventas_recientes.mean()
        
        # Pronosticar la demanda para el periodo de reposición
        pronostico = media_diaria * forecast_window
        
        resultados.append({
            'Codigo_Articulo': codigo,
            'Sucursal': sucursal,
            'Cantidad': round(pronostico, 2),
            'Average': round(media_diaria, 3)
        })

    # Crear el DataFrame de pronósticos
    df_pronostico = pd.DataFrame(resultados)

    # Exportar el resultado a un CSV para su posterior procesamiento
    df_pronostico.to_csv(f'data/{etiqueta}_Solicitudes_Compra.csv', index=False)

    return df_pronostico


def Calcular_Demanda_Extendida(df_prv,forecast_window,id_proveedor, etiqueta):
    max_date = df_prv['Fecha'].max()
    cutoff_date = max_date - timedelta(days=forecast_window)
    resultados_validacion = []

    # Agrupar los datos por 'Codigo_Articulo' y 'Sucursal'
    for (codigo, sucursal), grupo in df_prv.groupby(['Codigo_Articulo', 'Sucursal']):
        # Establecer 'Fecha' como índice y ordenar cronológicamente
        grupo = grupo.set_index('Fecha').sort_index()
        
        # Datos de entrenamiento: hasta la fecha de corte
        datos_entrenamiento = grupo.loc[:cutoff_date]
        
        # Verificar que existan suficientes datos para calcular el promedio (por ejemplo, al menos 30 días)
        if len(datos_entrenamiento) < 30:
            continue  # O vemos como manejarlo de otra forma
        
        # Resamplear a ventas diarias en el conjunto de entrenamiento
        ventas_diarias_entrenamiento = datos_entrenamiento['Unidades'].resample('D').sum().fillna(0)
        
        # Calcular la media diaria de los últimos 30 días del periodo de entrenamiento
        ventas_recientes = ventas_diarias_entrenamiento[-30:]
        media_diaria = ventas_recientes.mean()
        
        # Calcular el pronóstico para la ventana definida
        forecast = media_diaria * forecast_window
        
        # Definir el periodo de validación: desde el día siguiente a la fecha de corte hasta completar la ventana
        inicio_validacion = cutoff_date + timedelta(days=1)
        fin_validacion = cutoff_date + timedelta(days=forecast_window)
        
        # Extraer y resumir las ventas reales en el periodo de validación
        ventas_validacion = grupo.loc[inicio_validacion:fin_validacion]['Unidades'].resample('D').sum().fillna(0)
        ventas_reales = ventas_validacion.sum()
        
        # Calcular medidas de error
        error_absoluto = abs(forecast - ventas_reales)
        error_porcentual = (error_absoluto / ventas_reales * 100) if ventas_reales != 0 else None
        
        resultados_validacion.append({
            'Codigo_Articulo': codigo,
            'Sucursal': sucursal,
            'Forecast': round(forecast, 2),
            'Ventas_Reales': round(ventas_reales, 2),
            'Error_Absoluto': round(error_absoluto, 2),
            'Error_Porcentual': round(error_porcentual, 2) if error_porcentual is not None else None
        })

    # Crear un DataFrame con los resultados de validación
    df_validacion = pd.DataFrame(resultados_validacion)

    # Exportar el resultado a un CSV para su posterior procesamiento
    df_validacion.to_csv(f'data/{etiqueta}_Datos_Validacion.csv', index=False)

    return df_validacion


def Exportar_Pronostico(df_forecast, etiqueta, algoritmo):
    df_forecast['Codigo_Articulo']= df_forecast['Codigo_Articulo'].astype(int)
    df_forecast['Sucursal']= df_forecast['Sucursal'].astype(int)
    
    # tools.display_dataframe_to_user(name="SET de Datos del Proveedor", dataframe=df_forecast)
    # df_forecast.info()
    # print(f'data/{etiqueta}_Pronostico_{algoritmo}.csv')
    df_forecast.to_csv(f'data/{etiqueta}_Pronostico_{algoritmo}.csv', index=False)
    return


def Procesar_ALGO_05(data, proveedor, etiqueta, ventana, fecha):
    df_forecast = Calcular_Demanda(data, ventana, proveedor, etiqueta)
    df_validacion = Calcular_Demanda_Extendida(data, ventana, proveedor, etiqueta)
    Exportar_Pronostico(df_forecast, etiqueta, 'ALGO_05')  # Impactar Datos en la Interface
    print(f'ALGO_05 para: {etiqueta} ventana: {ventana}  - {fecha}')
    return

    




In [45]:
# RUTINA PRINCIPAL para obtener el pronóstico
def get_forecast(id_proveedor, lbl_proveedor, period_lengh = 30, algorithm='basic', current_date=None):
    """
    Permite la selección del algoritmo de predicción y calcula la demanda estimada.
    En el futuro podrémos ir agregando algoritmos

    Parámetros:
    - df: DataFrame con la información histórica.
    - algorithm: Algoritmo a utilizar (por defecto 'basic').
    - current_date: Fecha de referencia para el cálculo; si es None, se toma la fecha máxima del DataFrame.
    - period_length: Número de días del período a analizar.

    Retorna un DataFrame con las predicciones.
    """
    data, articulos = generar_datos(id_proveedor, lbl_proveedor)
    
    if current_date is None:
        current_date = data['Fecha'].max()
    else:
        current_date = pd.to_datetime(current_date)
        
    if algorithm == 'ALGO_05':
        forecast_df = Procesar_ALGO_05( data, id_proveedor, lbl_proveedor, period_lengh, current_date )
        
    else:
        raise ValueError(f"El algoritmo '{algorithm}' no está implementado.")

    return forecast_df

In [46]:
#EJECUCIÓN MASIVA x LISTA

proveedores = [
    # {"id": 20, "nombre": "MOLINOS RIO DE LA PLATA", "label": "20_MOLINOS", "ventana": 30, "algoritmo" : "ALGO_05"},
    # {"id": 25, "nombre": "CAFES LA VIRGINIA S.A.", "label": "25_LA_VIRGINIA", "ventana": 30, "algoritmo"  :  "ALGO_05"},
    # {"id": 62, "nombre": "ARCOR","label":"62_ARCOR", "label": "62_ARCOR", "ventana": 30, "algoritmo" : "ALGO_05"},
    {"id": 98, "nombre": "FRATELLI BRANCA DESTILERIAS S.A.", "label": "98_FRATELLI_BRANCA", "ventana": 30, "algoritmo" : "ALGO_05"},
    # {"id": 140, "nombre": "UNILEVER DE ARGENTINA S.A.", "label": "140_UNILEVER", "ventana": 30, "algoritmo" : "ALGO_05"},
    # {"id": 189, "nombre": "BODEGAS Y VIÑEDOS LOPEZ S.A.I.C.", "label": "189_BODEGAS_LOPEZ", "ventana": 30, "algoritmo" : "ALGO_05"},
    # {"id": 1465, "nombre": "QUICKFOOD S.A.", "label":"1465_QUICKFOOD", "ventana": 30, "algoritmo" : "ALGO_05"},
    #{"id": 327, "nombre": "PALADINI S.A.", "label":"327_PALADINI", "ventana": 30, "algoritmo" : "ALGO_05"}
]

for proveedor in proveedores:
    print(f'Procesando: {proveedor["id"]} - {proveedor["label"]} - {proveedor["ventana"]} - {proveedor["algoritmo"]}')

    get_forecast(proveedor["id"], proveedor["label"], proveedor["ventana"], proveedor["algoritmo"])
    print('------------------------------------------------------------------')
    
    
    


Procesando: 98 - 98_FRATELLI_BRANCA - 30 - ALGO_05
Generando datos para ID: 98, Label: 98_FRATELLI_BRANCA
Datos de Venta guardados: data/98_FRATELLI_BRANCA.csv
Archivos guardados: data/98_FRATELLI_BRANCA_articulos.csv
SET de Datos del Proveedor


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Codigo_Articulo  712 non-null    int32  
 1   Sucursal         712 non-null    int32  
 2   Cantidad         712 non-null    float64
 3   Average          712 non-null    float64
dtypes: float64(2), int32(2)
memory usage: 16.8 KB
data/98_FRATELLI_BRANCA_Pronostico_ALGO_05.csv
ALGO_05 para: 98_FRATELLI_BRANCA ventana: 30  - 2025-02-22 00:00:00
------------------------------------------------------------------


### VISUALIZACIÓN de DATOS DE EJECUCIÓN

In [ ]:
proveedor = 327
label = '327_Paladini'
forecast_window = 30

data, articulos = generar_datos(proveedor, label)
# Verificar después de la transformación
tools.display_dataframe_to_user(name="SET de Datos de VENTA", dataframe=data)
tools.display_dataframe_to_user(name="SET de ARTICULOS del Proveedor", dataframe=articulos)

# Calcular Demanda
df_forecast = Calcular_Demanda(data, forecast_window, proveedor, label)
tools.display_dataframe_to_user(name="Pronostico de Compras", dataframe=df_forecast)

# Calcular Demanda Extendida
df_validacion = Calcular_Demanda_Extendida(data, forecast_window, proveedor, label)
# Mostrar un resumen de los resultados de validación
tools.display_dataframe_to_user(name="SET de Validación de Datos de Compra", dataframe=df_validacion)

# REVISAR los 3 Archivos Generados en /data/


In [ ]:
#EJECUCIÓN DETALLADA
# Elegir el Proveedor
proveedor = 20
label = '20_Molinos'
forecast_window = 30  # o 45

data, articulos = generar_datos(proveedor, label)
# Verificar después de la transformación
tools.display_dataframe_to_user(name="SET de Datos de VENTA", dataframe=data)
tools.display_dataframe_to_user(name="SET de ARTICULOS del Proveedor", dataframe=articulos)

# Calcular Demanda
df_forecast = Calcular_Demanda(data, forecast_window, proveedor, label)
tools.display_dataframe_to_user(name="Pronostico de Compras", dataframe=df_forecast)

# Calcular Demanda Extendida
df_validacion = Calcular_Demanda_Extendida(data, forecast_window, proveedor, label)
# Mostrar un resumen de los resultados de validación
tools.display_dataframe_to_user(name="SET de Validación de Datos de Compra", dataframe=df_validacion)

# REVISAR los 3 Archivos Generados en /data/

In [ ]:
#Leer Datos de Ventas
# ELEGIR el PROVEEDOR
proveedor = 98
label = '98_Fratelli_Branca'
forecast_window = 30  # o 45

data, articulos = generar_datos(proveedor, label)
# Verificar después de la transformación
tools.display_dataframe_to_user(name="SET de Datos de VENTA", dataframe=data)
tools.display_dataframe_to_user(name="SET de ARTICULOS del Proveedor", dataframe=articulos)

# Calcular Demanda
df_forecast = Calcular_Demanda(data, forecast_window,proveedor, label)
tools.display_dataframe_to_user(name="Pronostico de Compras", dataframe=df_forecast)

# Calcular Demanda Extendida
df_validacion = Calcular_Demanda_Extendida(data, forecast_window,proveedor, label)
# Mostrar un resumen de los resultados de validación
tools.display_dataframe_to_user(name="SET de Validación de Datos de Compra", dataframe=df_validacion)

# REVISAR los 3 Archivos Generados en /data/